In [10]:
import tensorflow as tf

In [11]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=10000)

In [12]:
max_len = 512
num_classes = 2

X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [13]:
class Inception(tf.keras.Model):

    def __init__(self):
        super().__init__()

        self.p1_1 = tf.keras.layers.Conv1D(64, 1, activation='relu')

        self.p2_1 = tf.keras.layers.Conv1D(128, 1, activation='relu', dilation_rate=2)
        self.p2_2 = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu', dilation_rate=3)

        self.p3_1 = tf.keras.layers.Conv1D(128, 1, activation='relu')
        self.p3_2 = tf.keras.layers.Conv1D(256, 5, padding='same', activation='relu')
        self.p3_2 = tf.keras.layers.Conv1D(64, 5, padding='same', activation='relu')

        self.p4_1 = tf.keras.layers.MaxPool1D(3, 1, padding='same')
        self.p4_2 = tf.keras.layers.Conv1D(64, 1, activation='relu')

    def call(self, x):
        p1 = self.p1_1(x)
        p2 = self.p2_2(self.p2_1(x))
        p3 = self.p3_2(self.p3_1(x))
        p4 = self.p4_2(self.p4_1(x))

        return tf.keras.layers.Concatenate()([p1, p2, p3, p4])

In [14]:
model = tf.keras.Sequential()

model.add(tf.keras.layers.Embedding(input_dim=10000, output_dim=512))

model.add(tf.keras.layers.Conv1D(256, 1, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.MaxPool1D(pool_size=3, strides=2, padding='same'))

model.add(Inception())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.MaxPool1D(pool_size=3, strides=2, padding='same'))

model.add(tf.keras.layers.GlobalAvgPool1D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(2, activation="relu"))

In [15]:
input_shape=(512, 1)
model.build(input_shape)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'inception_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (512, 1, 512)               │       5,120,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_8 (Conv1D)                    │ (512, 1, 256)               │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (512, 1, 256)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (512, 1, 256)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_1 (Inception)              │ (512, 1, 256)               │         164,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (512, 1, 256)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_5 (MaxPooling1D)       │ (512, 1, 256)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling1d_1           │ (512, 256)                  │               0 │
│ (GlobalAveragePooling1D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (512, 256)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (512, 2)                    │             514 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,416,194 (20.66 MB)

 Trainable params: 5,416,194 (20.66 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
lr = 0.1
num_epochs = 10
batch_size = 128

In [17]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"])

model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    epochs=num_epochs,
    validation_split=0.1)

Epoch 1/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 18s 62ms/step - accuracy: 0.5550 - loss: 2.0913 - val_accuracy: 0.7708 - val_loss: 0.8801
Epoch 2/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 13s 32ms/step - accuracy: 0.7837 - loss: 0.5680 - val_accuracy: 0.8408 - val_loss: 0.6783
Epoch 3/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.8716 - loss: 0.3940 - val_accuracy: 0.8448 - val_loss: 0.7793
Epoch 4/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - accuracy: 0.8930 - loss: 0.3409 - val_accuracy: 0.8464 - val_loss: 0.9510
Epoch 5/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - accuracy: 0.8955 - loss: 0.3575 - val_accuracy: 0.8388 - val_loss: 0.8933
Epoch 6/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 10s 32ms/step - accuracy: 0.9094 - loss: 0.2923 - val_accuracy: 0.8340 - val_loss: 1.0567
Epoch 7/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.9227 - loss: 0.2598 - val_accuracy: 0.8408 - val_loss: 0.8742
Epoch 8/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.9231 - loss: 0.2768 - val

In [18]:
score = model.evaluate(X_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.2312363386154175
Test accuracy: 0.7996000051498413
